# Drizzle

Drizzle is a set of frontend libraries for ethereum. We use it in this example for building a web application that uses your blockchain accounts and network. We use a drizzle box to create a template that we base on our application. We should create a separate directory for the web app in the first place.

In [ ]:
!mkdir /home/codete/workshop/voting-drizzle

Generate a template app with drizzle is easy as below:

In [ ]:
!cd /home/codete/workshop/voting-drizzle && truffle unbox drizzle

The next step is to copy our contract to the ``contracts`` directory of the new app:

In [ ]:
!cp /home/codete/workshop/voting-truffle/contracts/Voting.sol /home/codete/workshop/voting-drizzle/contracts/

The box we used for our application use React as the library. There is one default component created in the templates called ``MyComponent.js``. It is used to display the data from our smart contract. We need to include the ``web3`` library that is used to convert the hex values returned by our smart contract into strings. We have two components imported that are used to invoke the methods from our smart contract (``ContractData``) and send data to our smart contract (``ContractForm``).

In [ ]:
%%writefile /home/codete/workshop/voting-drizzle/app/src/MyComponent.js
import React from "react";
import {
  AccountData,
  ContractData,
  ContractForm,
} from "drizzle-react-components";

import Web3 from 'web3';

var web3 = new Web3();

export default ({ accounts }) => (
  <div className="App">

    <div className="section">
      <h2>Active Account</h2>
      <AccountData accountIndex="0" units="ether" precision="3" />
    </div>

    <div className="VotingApp">
    <h2>Ideas</h2>
    <ContractData contract="Voting" method="getIdeas" render={data => (
        <ol> {console.log(data[0])}  
        {data[0].map((el,index)=>(
            <li>
                Title: {web3.utils.hexToString(el)}
                Description: {web3.utils.hexToString(data[1][index])}
            </li>
        ))}
    <h2>Vote for idea</h2>
    <ContractForm contract="Voting" method="addVote" labels={["Idea ID"]}/>
        </ol>
    )}/>
    <h2>Add a new idea</h2>
    <ContractForm 
      contract="Voting" 
      method="addIdea" 
      labels={["Title", "Description"]}
    />
    </div>
  </div>

);

The default container loads the default component and the contracts classes.

In [ ]:
%%writefile /home/codete/workshop/voting-drizzle/app/src/MyContainer.js
import MyComponent from "./MyComponent";
import { drizzleConnect } from "drizzle-react";

const mapStateToProps = state => {
  return {
    accounts: state.accounts,
    SimpleStorage: state.contracts.SimpleStorage,
    TutorialToken: state.contracts.TutorialToken,
    drizzleStatus: state.drizzleStatus,
    Voting: state.contracts.Voting,
  };
};

const MyContainer = drizzleConnect(MyComponent, mapStateToProps);

export default MyContainer;

Finally, we load the JSON compiled contracts and load the web3.

In [ ]:
%%writefile /home/codete/workshop/voting-drizzle/app/src/drizzleOptions.js
import SimpleStorage from "./contracts/SimpleStorage.json";
import ComplexStorage from "./contracts/ComplexStorage.json";
import TutorialToken from "./contracts/TutorialToken.json";
import Voting from "./contracts/Voting.json";

const options = {
  web3: {
    block: false,
    fallback: {
      type: "ws",
      url: "ws://127.0.0.1:9545",
    },
  },
  contracts: [SimpleStorage, ComplexStorage, TutorialToken, Voting],
  events: {
    SimpleStorage: ["StorageSet"],
  },
  polls: {
    accounts: 1500,
  },
};

export default options;

The last part is the configuration of truffle, similar as in the previous example, but this time we also need to set the path os the contracts.

In [ ]:
%%writefile /home/codete/workshop/voting-drizzle/truffle-config.js
const path = require("path");
  
module.exports = {
  contracts_build_directory: path.join(__dirname, "app/src/contracts"),
  networks: {
    development: {
      host: "127.0.0.1",      
      network_id: "*",            
      port: 8545
    }
  }
};

After building the contract we need to migrate it into our network:

In [ ]:
!cd /home/codete/workshop/voting-drizzle && truffle migrate

Ready to run the web app:

In [ ]:
!cd /home/codete/workshop/voting-drizzle/app && npm start

More templates can be found on the truffle framework website: https://truffleframework.com/boxes. More on the conversion can be found: https://web3js.readthedocs.io/en/1.0/web3-utils.html#hextoutf8